In [1]:
import xml.etree.ElementTree as ET

# Parse the XML data
def get_text(doc_path):
    tree = ET.parse(doc_path)
    root = tree.getroot()

    # Extract metadata
    metadata = root.find('./teiHeader')
    metadata_dict = {}
    for item in metadata.findall('metadata'):
        name = item.get('name')
        value = item.text
        metadata_dict[name] = value

    # Extract text content
    text = root.find('./text')
    body_author = text.find('./body').get('author')
    title_author = text.find('./title').get('author')
    sentences = text.findall('body/s')
    comments = text.findall('comment')
    comments_pairs = [([(word.get('type'), word.text) for word in c.findall('s/w')], c.get('c_type')) for c in comments]
    sentences_pairs = [[(word.get('type'), word.text) for word in sent.findall('w')] for sent in sentences]
    text = []
    for sentense in sentences_pairs:
        sentense_parsed = ''.join([word[1] for word in sentense])
        text.append(sentense_parsed)
    text = '\n'.join(text)
    return text


In [ ]:
import os
from tqdm.auto import tqdm

directory = '/nfs/nas-6.1/wclu/cllt/ptt_data/politics'
corpus = []
 
for root, dirs, files in os.walk(directory):
    for filename in tqdm(files):
        if filename != '.DS_Store':
            doc_path = os.path.join(root, filename)
            corpus.append(get_text(doc_path))


## BM25

In [3]:
from ckip_transformers.nlp import CkipWordSegmenter
ws_driver  = CkipWordSegmenter(model="bert-base", device=0)
corpus_tokenized = ws_driver(corpus, batch_size=128, max_length=509)

2023-05-25 13:17:25.755255: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-25 13:17:25.839942: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 13:17:46.865273: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Inference: 100%|██████████| 2/2 [00:05<00:00,  2.77s/it]


In [4]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(corpus_tokenized)

In [7]:
def retrieve_bm25(query):
    tokenized_query = ws_driver([query], batch_size=1, max_length=509)[0]
    rel_doc = bm25.get_top_n(tokenized_query, corpus, n=10)
    return rel_doc


In [11]:
query = '侯友宜跟賴清德的比較'
retrieve_bm25(query)

Inference: 100%|██████████| 1/1 [00:00<00:00, 46.18it/s]


['棄中保賴？綠營小雞高掛「信賴台灣」王郁揚：新潮流與傑太日煙關係匪淺\nhttps://reurl.cc/RXMLo6\n台灣威卜 VAPE TAIWAN\nhttps://i.imgur.com/WCvRrJM.jpg\n金色力量王郁揚發現民進黨議員候選人紛紛掛起與副總統賴清德的競選看板，\n「棄中保賴」的味道濃郁。/圖：台灣威卜VAPE TAIWAN 合成。\n11月26日九合一選舉進入白熱化的同時，台北市士林北投區驚見民進黨小雞高掛與副總統\n賴清德合照看板主打「 信賴台灣 」，金色力量士林北投議員候選人王郁揚表示，英系陳\n時中台北市長民調持續落後，民進黨小雞為求自保分打出「棄中保賴」，而我要提醒大家\n留意「民進黨新潮流與傑太日煙的關係匪淺」，2022、2024下架新潮流！\n陳時中民調落後 「抗中保台」神主牌將登場救援？\nTVBS日前公布最新民調，國民黨台北市長參選人蔣萬安的支持度增加4個百分點，來到40%\n，第二名為無黨籍的黃珊珊，支持度為23%，民進黨籍的陳時中則下滑到第三位，支持度為\n22%。民進黨前立委郭正直言，綠營現在思考的是市議員不要選太差，已經不太去想陳時中\n是否會當選，「如果選到第三，市議員會很慘」，因此又拿出「抗中保台」想要鞏固基本\n盤。\n柯文哲曾嗆：新潮流不倒 台灣不會好\nETTODAY 報導，大巨蛋案爭議延燒時，民進黨議員簡舒培提出民眾爆料的會議錄音檔，引\n來備詢台上的柯文哲怒斥，這是抹黑的手法，錄音90分鐘剪7段，為什麼不全部放？仍不認\n為是會議而是私下討論，痛批議員抹黑、含沙射影，「新潮流不倒台灣不會好！」\nhttps://youtu.be/mPPQZv4TJuo\n最新》批簡舒培錄音檔抹黑 柯P怒：新潮流不倒 台灣不會好。影：東森新聞 CH51。\n金色力量黨主席王郁揚：2022、2024下架新潮流！\n金色力量黨主席同時也是全台唯一取得世界衛生組織授證的菸草減害(Tobacco Harm\nReduction ,THR)專家 王郁揚表示，我曾於2018年投書媒體「再次」揭露民進黨新潮流與\n傑太日煙(JTI, Japan Tobacco International)的「關係匪淺」，除了蔡英文總統專機走\n私菸案(傑太菸品走私最多)、更有副總統賴清德同意設立的台南JTI菸廠、監察院長陳菊也\n曾因為菸捐訪

## Semantic Search

In [5]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('GanymedeNil/text2vec-base-chinese', device=2 ,cache_folder='/nfs/nas-6.1/wclu/cache')

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True, normalize_embeddings=True, device=2, show_progress_bar=True)

No sentence-transformers model found with name /nfs/nas-6.1/wclu/cache/GanymedeNil_text2vec-base-chinese. Creating a new one with MEAN pooling.
2023-05-25 14:21:22.926444: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-25 14:21:22.999091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 14:21:26.386794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def retrieve_semantic(query):
    query_embedding = embedder.encode([query], convert_to_tensor=True, normalize_embeddings=True, device=2, show_progress_bar=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, score_function=util.dot_score, top_k=10)
    rel_doc = [corpus[hit['corpus_id']] for hit in hits[0]]
    return rel_doc

In [8]:
query = '侯友宜跟賴清德的比較'
retrieve_semantic(query)

['如果2022年的台北市長選戰\n最後是由蔣萬安vs黃珊珊vs陳時中\n大家覺得勝負如何\n個人覺得會重演當年趙少康vs黃大洲vs陳水扁的結果的機率很大\n不知大家怎看?',
 '如題，最近的監察院長可謂鬧得沸沸揚揚不可開交，也看到許多笑話例如阿珍偷桃；費費\n掛彩；嘉瑜完腳。這些亂象我想台灣人民已經見怪不怪。但國民主張廢考監，但做的與講\n的南轅北轍。而國民黨激烈的抗爭與破壞公物，是非常不好的示範，而且讓大家完全失焦\n在國民黨荒腔走板的行為，反而讓陳菊暗渡陳倉。這不是爽到陳菊嗎？ 再來，如果能讓\n陳菊好好備詢，就算被問到祖宗18代，她只要打哈哈 敷衍，民進黨狹著多數黨優勢還是\n通過阿！！\n所以 罪魁禍首就是 黑台灣錢護台灣權的民進黨。\n台灣又不是只有陳菊是人權鬥士，也不是只有她為了台灣的民主奮鬥啊，在座的各位也每\n一天啊都為台灣民主奮鬥啊。',
 '棄中保賴？綠營小雞高掛「信賴台灣」王郁揚：新潮流與傑太日煙關係匪淺\nhttps://reurl.cc/RXMLo6\n台灣威卜 VAPE TAIWAN\nhttps://i.imgur.com/WCvRrJM.jpg\n金色力量王郁揚發現民進黨議員候選人紛紛掛起與副總統賴清德的競選看板，\n「棄中保賴」的味道濃郁。/圖：台灣威卜VAPE TAIWAN 合成。\n11月26日九合一選舉進入白熱化的同時，台北市士林北投區驚見民進黨小雞高掛與副總統\n賴清德合照看板主打「 信賴台灣 」，金色力量士林北投議員候選人王郁揚表示，英系陳\n時中台北市長民調持續落後，民進黨小雞為求自保分打出「棄中保賴」，而我要提醒大家\n留意「民進黨新潮流與傑太日煙的關係匪淺」，2022、2024下架新潮流！\n陳時中民調落後 「抗中保台」神主牌將登場救援？\nTVBS日前公布最新民調，國民黨台北市長參選人蔣萬安的支持度增加4個百分點，來到40%\n，第二名為無黨籍的黃珊珊，支持度為23%，民進黨籍的陳時中則下滑到第三位，支持度為\n22%。民進黨前立委郭正直言，綠營現在思考的是市議員不要選太差，已經不太去想陳時中\n是否會當選，「如果選到第三，市議員會很慘」，因此又拿出「抗中保台」想要鞏固基本\n盤。\n柯文哲曾嗆：新潮流不倒 台灣不會好\nETTODAY 報導，大巨蛋案爭議延燒時，民進黨議員簡舒培提出民眾爆料的會議錄音檔，引\n